In [29]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import itertools
from sklearn.metrics import mean_squared_error, r2_score

In [30]:
df = pd.read_csv("California_Houses.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Median_House_Value        20640 non-null  float64
 1   Median_Income             20640 non-null  float64
 2   Median_Age                20640 non-null  int64  
 3   Tot_Rooms                 20640 non-null  int64  
 4   Tot_Bedrooms              20640 non-null  int64  
 5   Population                20640 non-null  int64  
 6   Households                20640 non-null  int64  
 7   Latitude                  20640 non-null  float64
 8   Longitude                 20640 non-null  float64
 9   Distance_to_coast         20640 non-null  float64
 10  Distance_to_LA            20640 non-null  float64
 11  Distance_to_SanDiego      20640 non-null  float64
 12  Distance_to_SanJose       20640 non-null  float64
 13  Distance_to_SanFrancisco  20640 non-null  float64
dtypes: flo

In [31]:
X = df.drop("Median_House_Value", axis=1)
y = df["Median_House_Value"]

In [32]:

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42
)
X_val, X_test, y_val,y_test = train_test_split(
     X_temp, y_temp, test_size=0.50, random_state=42
)
total_rows = len(df)
print(f"Total rows: {total_rows}")
print(f"Training set:   {len(X_train)} rows ({len(X_train)/total_rows:.2%})")
print(f"Validation set: {len(X_val)} rows ({len(X_val)/total_rows:.2%})")
print(f"Testing set:    {len(X_test)} rows ({len(X_test)/total_rows:.2%})")

Total rows: 20640
Training set:   14448 rows (70.00%)
Validation set: 3096 rows (15.00%)
Testing set:    3096 rows (15.00%)


In [33]:
# scaling the data first so could apply regularization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)
y_train = y_train / 100000
y_val = y_val / 100000
y_test = y_test / 100000


In [34]:
class CustomRegressor:
    def __init__(self, X, y, epochs=100, penalty=None,  gradient_method="batch"):

        self.epochs = epochs
        self.penalty=penalty

        self.weights = None
        self.bias = None
        self.X = X
        self.y = y
        self.gradient_method = gradient_method
    def fit(self,learning_rate=0.01, lambd=0):
        self.lambd=lambd
        self.learning_rate=learning_rate
        n_samples, n_features = self.X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        for i in range(self.epochs):
            if self.gradient_method == "batch":
                y_predicted = np.dot(self.X, self.weights) + self.bias
                dw = (1/n_samples) * np.dot(self.X.T, (y_predicted - self.y)) # X.T is the xj in the gradient descent(row reppresent feature and coluumn represent sample) as now each row in X.T represent feature, and as they are vectors the loop is implicity inside them
                db = (1/n_samples) * np.sum(y_predicted - self.y)

                if self.penalty == "l2":
                    dw += 2 * self.lambd * (self.weights)
                elif self.penalty == "l1":
                    dw += (self.lambd/ n_samples) * np.sign(self.weights) # diffrentiate the w multiplied with constant so remain the constant with the sign of w

                self.weights -= self.learning_rate * dw
                self.bias  -= self.learning_rate * db
            elif self.gradient_method == "sgd":
                indices = np.random.permutation(n_samples)
                for idx in indices:
                    xi = self.X[idx:idx+1]
                    yi = self.y.iloc[idx]
                    y_pred = np.dot(xi, self.weights) + self.bias
                    dw = np.dot(xi.T, (y_pred - yi)).flatten()
                    db = (y_pred - yi).item()

                    if self.penalty == "l2":
                        dw +=  (2*self.lambd/ n_samples) * (self.weights)
                    elif self.penalty == "l1":
                        dw += (self.lambd/n_samples) * np.sign(self.weights)

                    self.weights -= self.learning_rate * dw
                    self.bias -= self.learning_rate * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

In [35]:
learning_rates = [0.001, 0.0001, 0.00001, 0.000001]
lambdas = [0, 0.001, 0.01, 0.1, 1, 10, 100]

In [36]:



def grid_search(X_train, y_train, X_val, y_val, penalty_type, optimizer_type):
    print(f"--- Tuning {penalty_type} ({optimizer_type}) ---")

    best_mse = float('inf')
    best_params = {}

    model = CustomRegressor(X_train, y_train, 1, penalty_type, optimizer_type)

    for lr, l in itertools.product(learning_rates, lambdas):
        model.fit(lr, l)
        preds = model.predict(X_val)
        mse = mean_squared_error(y_val, preds)

        if mse < best_mse:
            best_mse = mse
            best_params = {'learning_rate': lr, 'lambd': l}

    if penalty_type == None:
        best_params['lambd'] = 0

    print(f"Best Params: {best_params}")
    print(f"Best Validation MSE: {best_mse:.2f}")
    print("-" * 30)
    return best_params


In [37]:
best_ridge_batch = grid_search(X_train_scaled, y_train, X_val_scaled, y_val, "l2", "batch")
best_lasso_batch = grid_search(X_train_scaled, y_train, X_val_scaled, y_val, "l1", "batch")

best_ridge_sgd = grid_search(X_train_scaled, y_train, X_val_scaled, y_val, "l2", "sgd")

best_linear_batch = grid_search(X_train_scaled, y_train, X_val_scaled, y_val, None, "batch")

best_linear_sgd = grid_search(X_train_scaled, y_train, X_val_scaled, y_val, None, "sgd")



best_lasso_sgd = grid_search(X_train_scaled, y_train, X_val_scaled, y_val, "l1", "sgd")



--- Tuning l2 (batch) ---
Best Params: {'learning_rate': 0.001, 'lambd': 0}
Best Validation MSE: 5.57
------------------------------
--- Tuning l1 (batch) ---
Best Params: {'learning_rate': 0.001, 'lambd': 0}
Best Validation MSE: 5.57
------------------------------
--- Tuning l2 (sgd) ---
Best Params: {'learning_rate': 0.001, 'lambd': 10}
Best Validation MSE: 0.51
------------------------------
--- Tuning None (batch) ---
Best Params: {'learning_rate': 0.001, 'lambd': 0}
Best Validation MSE: 5.57
------------------------------
--- Tuning None (sgd) ---
Best Params: {'learning_rate': 0.001, 'lambd': 0}
Best Validation MSE: 0.50
------------------------------
--- Tuning l1 (sgd) ---
Best Params: {'learning_rate': 0.001, 'lambd': 0}
Best Validation MSE: 0.51
------------------------------


In [38]:
final_configs = [
    ("Linear Regression (Batch)", best_linear_batch, None, "batch"),
    ("Linear Regression (SGD)",   best_linear_sgd,   None, "sgd"),
    ("Ridge Regression (Batch)",  best_ridge_batch,  "l2", "batch"),
    ("Ridge Regression (SGD)",    best_ridge_sgd,    "l2", "sgd"),
    ("Lasso Regression (Batch)",  best_lasso_batch,  "l1", "batch"),
    ("Lasso Regression (SGD)",    best_lasso_sgd,    "l1", "sgd"),
]

results_data = []

print("--- FINAL TESTING RESULTS ---\n")

# 2. Loop through each configuration
for name, params, penalty, optimizer in final_configs:


    model = CustomRegressor(
        X_train_scaled,
        y_train,
        epochs=100,
        penalty=penalty,
        gradient_method=optimizer
    )


    model.fit(params['learning_rate'],params['lambd'])


    y_pred_test = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred_test)
    r2 = r2_score(y_test, y_pred_test)

    results_data.append({
        "Model": name,
        "Optimizer": optimizer.upper(),
        "Best Lambda": params['lambd'],
        "Best LR": params['learning_rate'],
        "Test MSE": mse,
        "Test R2": r2
    })

    print(f"{name}: MSE={mse:.2f}, R2={r2:.4f}")



results_df = pd.DataFrame(results_data)

print("\n--- LEADERBOARD ---")
display(results_df.sort_values(by="Test MSE", ascending=True))

--- FINAL TESTING RESULTS ---

Linear Regression (Batch): MSE=4.63, R2=-2.4999
Linear Regression (SGD): MSE=0.44, R2=0.6664
Ridge Regression (Batch): MSE=4.63, R2=-2.4999
Ridge Regression (SGD): MSE=0.44, R2=0.6646
Lasso Regression (Batch): MSE=4.63, R2=-2.4999
Lasso Regression (SGD): MSE=0.45, R2=0.6614

--- LEADERBOARD ---


,Model,Optimizer,Best Lambda,Best LR,Test MSE,Test R2
1,Linear Regression (SGD),SGD,0,0.001,0.441074,0.666437
3,Ridge Regression (SGD),SGD,10,0.001,0.443509,0.664595
5,Lasso Regression (SGD),SGD,0,0.001,0.447688,0.661435
0,Linear Regression (Batch),BATCH,0,0.001,4.627913,-2.499869
2,Ridge Regression (Batch),BATCH,0,0.001,4.627913,-2.499869
4,Lasso Regression (Batch),BATCH,0,0.001,4.627913,-2.499869
